In [82]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.feature_selection import RFE
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor

In [11]:
df = pd.read_csv("数据.csv" , encoding = 'gbk')
df.head(5)

,预警状态,路段平均速度,行驶时间,方向角变化,高程变化,平均加速度,温度3,相对湿度3,能见度3,天气情况3,时间分段4,方向角标准差,高程标准差
0,0,24.750000,16.0,0.568750,0.066667,2.847222,1,3,3,3,2,0.629782,0.103638
1,0,31.384615,26.0,0.690385,0.114744,1.538051,2,3,3,3,2,0.569916,0.091083
2,0,67.028571,35.0,0.392857,0.253333,0.620635,2,2,1,3,3,0.427588,0.193402
3,0,24.461538,13.0,0.600000,0.124359,2.370151,3,2,1,3,3,1.284523,0.096132
4,0,20.285714,21.0,0.111905,0.140476,1.654699,2,2,1,3,3,0.114822,0.084695


In [12]:
# 设置哑变量
df = df.join(pd.get_dummies(df['温度3'] , prefix = 'dum温度'))
df = df.join(pd.get_dummies(df['相对湿度3'] , prefix = 'dum相对湿度'))
df = df.join(pd.get_dummies(df['能见度3'] , prefix = 'dum能见度'))
df = df.join(pd.get_dummies(df['天气情况3'] , prefix = 'dum天气情况'))
df = df.join(pd.get_dummies(df['时间分段4'] , prefix = 'dum时间分段'))
df.head(5)

,预警状态,路段平均速度,行驶时间,方向角变化,高程变化,平均加速度,温度3,相对湿度3,能见度3,天气情况3,...,dum能见度_1,dum能见度_2,dum能见度_3,dum天气情况_1,dum天气情况_2,dum天气情况_3,dum时间分段_1,dum时间分段_2,dum时间分段_3,dum时间分段_4
0,0,24.750000,16.0,0.568750,0.066667,2.847222,1,3,3,3,...,0,0,1,0,0,1,0,1,0,0
1,0,31.384615,26.0,0.690385,0.114744,1.538051,2,3,3,3,...,0,0,1,0,0,1,0,1,0,0
2,0,67.028571,35.0,0.392857,0.253333,0.620635,2,2,1,3,...,1,0,0,0,0,1,0,0,1,0
3,0,24.461538,13.0,0.600000,0.124359,2.370151,3,2,1,3,...,1,0,0,0,0,1,0,0,1,0
4,0,20.285714,21.0,0.111905,0.140476,1.654699,2,2,1,3,...,1,0,0,0,0,1,0,0,1,0


In [34]:
trainset = df[['路段平均速度','行驶时间' , '方向角变化' , '高程变化' , '平均加速度' , '方向角标准差','高程标准差','dum温度_1','dum温度_2',\
              'dum温度_3','dum相对湿度_1','dum相对湿度_2','dum相对湿度_3','dum能见度_1','dum能见度_2','dum能见度_3','dum天气情况_1',\
              'dum天气情况_2','dum天气情况_3','dum时间分段_1','dum时间分段_2','dum时间分段_3','dum时间分段_4']].values
target = list(df['预警状态'])

In [39]:
min_max_scaler = preprocessing.MinMaxScaler()
trainset_minmax = min_max_scaler.fit_transform(trainset)

**svm各个feature的特征重要性排序**

下面输出了各个特征的重要性排序，1就是最终要的特征，依次类推。我这边没有使用z-score归一化,使用的是min max归一化，这种方式是可以对分类和连续变量都做的，所以我认为在这种归一化下分类变量和连续变量的重要性也是也是可以比较的。没有参考文献，是我自己的理解。（你也可以问问你导师的看法，如果你能把你导师的意见告诉我下，将不胜感激，hhh）其实不严格的说，哪怕做了z-score，你直接把分类变量和连续变量进行重要性比较问题也不大，基本也可以认为是准的，当然从推导来说，我觉得不严谨。

In [79]:
# 使用svm-RBF做特征重要性排序
estimator = SVC(kernel="linear")
selector = RFE(estimator, 1, step=1)
selector = selector.fit(trainset_minmax, target)
print("The feature that has been selected" , selector.support_)  # Get a mask, or integer index, of the features selected
print("selected features' rankings" , selector.ranking_)

The feature that has been selected [False False False False False False False False False False False False
 False False False False False  True False False False False False]
selected features' rankings [13  7  8 12  6  4  5 21 23 16 19 11 10 15 22 18  2  1  3 17 14 20  9]


**svm做预测**

现在我能理解为什么你的论文里是直接做预测的，因为你看重的各个特征的排序效果，而不是预测结果，所以可能就没那么严格用测试集、训练集来搞了，我这边就按照你的那篇参考文献来了。下面的svm模型做了特征选择，最后是留下了一半的feature做预测的结果

In [80]:
# 使用svm-RBF做特征重要性排序
estimator = SVC(C=1.0,kernel="linear")
selector = RFE(estimator,step=1)
selector = selector.fit(trainset_minmax, target)
print("The feature that has been selected" , selector.support_)  # Get a mask, or integer index, of the features selected
print("selected features' rankings" , selector.ranking_)

The feature that has been selected [False  True  True False  True  True  True False False False False  True
  True False False False  True  True  True False False False  True]
selected features' rankings [ 3  1  1  2  1  1  1 11 13  6  9  1  1  5 12  8  1  1  1  7  4 10  1]


In [81]:
selector.score(trainset_minmax , target)

0.6877323420074349

**随机森林各个feature的特征重要性排序**

In [104]:
# 使用svm-RBF做特征重要性排序
estimator = RandomForestRegressor(n_estimators=100)
selector = RFE(estimator, 1, step=1)
selector = selector.fit(trainset_minmax, target)
print("The feature that has been selected" , selector.support_)  # Get a mask, or integer index, of the features selected
print("selected features' rankings" , selector.ranking_)

The feature that has been selected [False False False False False  True False False False False False False
 False False False False False False False False False False False]
selected features' rankings [ 2  7  5  3  4  1  6 11 15  9 22 17 13 10 21 19 23 18 16  8 20 14 12]


**随机森林做预测**

In [103]:
# 使用svm-RBF做特征重要性排序
estimator = RandomForestRegressor(n_estimators=100)
selector = RFE(estimator,step=1)
selector = selector.fit(trainset_minmax, target)
print("The feature that has been selected" , selector.support_)  # Get a mask, or integer index, of the features selected
print("selected features' rankings" , selector.ranking_)

The feature that has been selected [ True  True  True  True  True  True  True  True False  True False False
 False  True False False False False False  True False False False]
selected features' rankings [ 1  1  1  1  1  1  1  1  6  1 10 11  2  1 12  9 13  7  5  1  8  4  3]


In [102]:
selector.score(trainset_minmax , target)

0.8782949195484354